In [4]:
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import mysql.connector

# 配置数据库连接
db_config = {
    'user': 'root',
    'password': 'nx123456789lsx',
    'host': 'localhost',
    'database': 'ocean_data'
}

# 创建数据库连接
conn = mysql.connector.connect(**db_config)
cursor = conn.cursor()
# 使用SQLAlchemy创建引擎
engine = create_engine(
    f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

In [40]:
# 创建表（如果尚未创建）
cursor.execute('''
CREATE TABLE IF NOT EXISTS location (
    location_id INT AUTO_INCREMENT PRIMARY KEY,
    latitude DOUBLE,
    longitude DOUBLE
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS observation (
    observation_id INT AUTO_INCREMENT PRIMARY KEY,
    location_id INT,

    depth DOUBLE,
    temperature DOUBLE,
    salinity DOUBLE,
    density DOUBLE,
    FOREIGN KEY (location_id) REFERENCES location (location_id)
);
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS gradient (
    gradient_id INT AUTO_INCREMENT PRIMARY KEY,
    observation_id INT,
    temperature_gradient DOUBLE,
    salinity_gradient DOUBLE,
    density_gradient DOUBLE,
    FOREIGN KEY (observation_id) REFERENCES observation (observation_id)
);
''')

In [2]:
# 读取CSV文件
temperature_df = pd.read_csv('../data/temperature.csv', skiprows=[1]).drop('depth',axis=1)
salinity_df = pd.read_csv('../data/salinity.csv', skiprows=[1]).drop('depth',axis=1)
density_df = pd.read_csv('../data/density.csv', skiprows=[1]).drop('depth',axis=1)

# 转换时间为日期格式
temperature_df['time'] = pd.to_datetime(temperature_df['time']).dt.date
salinity_df['time'] = pd.to_datetime(salinity_df['time']).dt.date
density_df['time'] = pd.to_datetime(density_df['time']).dt.date

# 合并数据
merged_df = temperature_df.merge(salinity_df, on=[
                                 'time', 'longitude', 'latitude'], how="outer")
merged_df = merged_df.merge(
    density_df, on=['time', 'longitude', 'latitude'], how="outer")
merged_df.rename(columns={'T': 'temperature',
                 'Salinity': 'salinity', 'Density': 'density'}, inplace=True)

In [41]:
# 插入location数据
location = merged_df[['latitude', 'longitude']
                     ].drop_duplicates().reset_index(drop=True)


location.to_sql('location', con=engine, if_exists='append', index=False)

203273

In [5]:
# 获取插入后的location_id
location = pd.read_sql('SELECT * FROM location', con=engine)

# 生成location_id
merged_df = merged_df.merge(location, on=['latitude', 'longitude'], how='left')

In [6]:
merged_df

,time,longitude,latitude,temperature,salinity,density,location_id
0,2024-06-03,99.0,52.000,NaN,NaN,NaN,1
1,2024-06-03,99.0,51.875,NaN,NaN,NaN,2
2,2024-06-03,99.0,51.750,NaN,NaN,NaN,3
3,2024-06-03,99.0,51.625,NaN,NaN,NaN,4
4,2024-06-03,99.0,51.500,NaN,NaN,NaN,5
...,...,...,...,...,...,...,...
1016360,2024-06-08,150.0,-9.500,28.430012,34.794113,1022.10095,203269
1016361,2024-06-08,150.0,-9.625,28.500020,34.792810,1022.07684,203270
1016362,2024-06-08,150.0,-9.750,28.800040,34.801810,1021.98420,203271
1016363,2024-06-08,150.0,-9.875,28.970024,34.799470,1021.92584,203272


In [7]:
# 插入observations数据
observation = merged_df[['location_id', 'time',
                         'temperature', 'salinity', 'density']]
observation.rename(columns={'time': 'date'}, inplace=True)
observation.to_sql('observation', con=engine, if_exists='append', index=False, dtype={
    'temperature': sqlalchemy.DOUBLE, 'salinity': sqlalchemy.DOUBLE, 'density': sqlalchemy.DOUBLE})

# 关闭数据库连接
conn.close()

C:\Users\nanxi\AppData\Local\Temp\ipykernel_73692\1761057925.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  observation.rename(columns={'time': 'date'}, inplace=True)
